# **Exercises for Dimensionality Reduction: PCA** [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/trendinafrica/Comp_Neuro-ML_course/tree/main/notebooks/19-Monday/Dimensionality_reduction.ipynb)
### **TReND Computational Neuroscience and Machine Learning Summer School, 2023**
#### Byron Yu and Joana Soldado Magraner, Carnegie Mellon University

---

The dataset for the following problems is contained in data_for_exercises.mat, on our Google Drive. You will find the following variable:

Xplan: a 728 x 97 matrix of real data, where each row is a spike count vector across 97 simultaneously-recorded neurons in dorsal premotor cortex (PMd) of a macaque monkey$^1$. Spike counts are taken in a 200 ms bin while the monkey is planning to make an arm reach. There are 91 trials for each of 8 reaching angles, for a total of 728 trials. Trials 1 to 91 correspond to reaching angle 1, trials 92 to 182 correspond to reach angle 2, etc.

1. The neural data have been generously provided by the laboratory of Prof. Krishna Shenoy at Stanford University. The data are to be used exclusively for educational purposes in this course.

#### Run the following cell to import all dependencies

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ipympl
%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import linalg
import scipy.io as spio

from google.colab import output
output.enable_custom_widget_manager()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.8 MB/s eta 0:00:00



#### Load the data



In [3]:
# Import data from googledrive
datapath  = '/content/drive/MyDrive/Soldado-Magraner/TReND_CompNeuro/Teaching Materials/19-Monday/'
datafile  = 'data_for_exercises.mat'

# Load the Matlab format .mat data into Python
mat   = spio.loadmat(datapath + datafile, squeeze_me=True)
Xplan = mat['Xplan']

plan_data_dims = Xplan.shape

print('dims = (N=samples/datapoints, D=neurons/features)')
print(plan_data_dims)

FileNotFoundError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Visualization of high-dimensional neural activity using PCA

We will apply PCA to Xplan to gain some intuition about plan activity in PMd. The data points are $\boldsymbol{x}_n \in \mathbb{R}^D$ (n = 1, ..., N), where D = 97 is the data dimensionality
and N = 728 is the number of data points.

[EXERCISE 1a] Run PCA on the high-dimensional Xplan data. You can do this by simply diagonalizing the covariance matrix of the data.

In [ ]:
# first find the mean of the data
mu = np.mean(Xplan,axis=0)
n  = Xplan.shape[0]
d  = Xplan.shape[1]

# now compute the data covariance
Xplan_centered = ... # don't forget to center your data first!
cov = ...            # now you can compute the covariance

# perform the eigendecomposition of the data covariance
D, U = ... # D = vector of eigenvalues, U = eigenvectors matrix

# make sure the eigenvalues are sorted (in descending order)
indx = np.argsort(D)[::-1]
D    = D[indx]

# arrange the eigenvectors according to the magnitude of the eigenvalues
U = U[:,indx]

In [ ]:
#@title Double click to see solution {display-mode: "form" }
# first find the mean of the data
mu = np.mean(Xplan,axis=0)
n  = Xplan.shape[0]
p  = Xplan.shape[1]

# now compute the data covariance
Xplan_centered = Xplan-np.tile(mu, (n, 1))  # don't forget to center your data first!
cov = Xplan_centered.T @ Xplan_centered     # now you can compute the covariance

# perform eigendecomposition of the data covariance
D, U = np.linalg.eig(cov) # D = vector of eigenvalues, U = eigenvectors matrix

# make sure the eigenvalues are sorted (in descending order)
indx = np.argsort(D)[::-1]
D    = D[indx]

# arrange the eigenvectors according to the magnitude of the eigenvalues
U = U[:,indx]

Plot the square-rooted eigenvalue spectrum. If you had to identify an elbow in the eigenvalue spectrum, how many dominant eigenvalues would there be?

In [ ]:
#plot the square-root eigenvalue spectrum
plt.plot(np.sqrt(D),'-o',color='black')
plt.xlabel('dimensions')
plt.ylabel('square-rooted eigenvalues')
plt.title('Square-root eigenvalue spectrum')
plt.show()

Note that there is an elbow (a big drop) after the 3rd eigenvalue. This indicates that the first three dimensions capture most of the data variance.

[EXERCISE 1b] What percentage of the overall data variance is captured by the top 3 principal components?

In [ ]:
# Hint, this is a function of the eigenvectors D
percent_var = ....
print('percent variance explained = ', percent_var)

In [ ]:
#@title Double click to see solution {display-mode: "form" }
# Hint, this is a function of the eigenvectors D
percent_var = 100 * np.sum(D[0:3])/np.sum(D)
print('percent variance explained = ', percent_var)

For the purposes of visualization, consider the PC space defined by the 3 eigenvectors.

[EXERCISE 1c] Project the data into the three-dimensional PC space. Plot the projected points, and color each dot appropriately according to reaching angle (there should be a total of 728 dots). Use your mouse to rotate the three-dimensional plot. Show a view in which the clusters are well-separated

In [ ]:
colors = ['r','k','y','g','b','m','c','darkviolet']
num_reaches_per_angle = 91;

fig = plt.figure()
ax  = fig.add_subplot(projection='3d')

for reachAngle in range(8):
  indx = np.arange(0,num_reaches_per_angle,1)+(reachAngle)*num_reaches_per_angle
  X = Xplan[indx,:]
  ax.scatter(X@U[:,0],X@U[:,1],X@U[:,2],'o', color=colors[reachAngle])

ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_zlabel('PC 3')

plt.title('Reach data projected into three-dimensional PC space')


[EXERCISE 1d] Define a matrix $U_M$ containing the top 3 eigenvectors (i.e. the top 3 PCs), where $U(d,m)$ indicates the contribution of the dth neuron to the mth principal component. Show the values in $U$ as a heatmap. Are there are any obvious groupings among the neurons in each column of $U$?

In [ ]:
fig = plt.figure()
U3  = ...
pos=plt.imshow(U3, aspect='auto')
fig.colorbar(pos)
plt.xticks([0,1,2],[1,2,3])
plt.xlabel('Principal component #')
plt.ylabel('Neuron #')
plt.title('Heat map of top three principal component directions')

In [ ]:
#@title Double click to see solution {display-mode: "form" }
fig = plt.figure()
U3  = U[:,0:3]
pos=plt.imshow(U3, aspect='auto')
fig.colorbar(pos)
plt.xticks([0,1,2],[1,2,3])
plt.xlabel('Principal component #')
plt.ylabel('Neuron #')
plt.title('Heat map of top three principal component directions')

Most elements of the top three eigenvectors are relatively small, with absolute values less than 0.15, indicating that those neurons contibute relatively little to the corresponding principal component. The relatively large elements of a particular eigenvector, with absolute values greater than 0.15, indicate clusters of neurons working together along the direction defined by the eigenvector.